In [1]:
%matplotlib inline
from pymongo import MongoClient

import math
from matplotlib import pyplot as plt
import seaborn as sns
import gpxpy.geo

In [2]:
client = MongoClient()
db = client.yelp

In [3]:
that = db.businesses.find()

In [4]:
len(list(that))

144072

In [17]:
def find_competitors_in_region(business_id, r):
    """r is in miles"""
    business = db.businesses.find_one({"business_id" : business_id})
    latitude, longitude = float(business['latitude']), float(business['longitude'])
    lat = math.radians(latitude)
    lon = math.radians(longitude)

    radius  = 3959.
    # Radius of the parallel at given latitude
    parallel_radius = radius*math.cos(lat)

    lat_min = math.degrees(lat - r/radius)
    lat_max = math.degrees(lat + r/radius)
    lon_min = math.degrees(lon - r/parallel_radius)
    lon_max = math.degrees(lon + r/parallel_radius)
    
    print lat_min, lat_max, lon_min, lon_max
    
    pipe = {"latitude": {"$gte": lat_min, "$lt": lat_max},
            "longitude": {"$gte": lon_min, "$lt": lon_max},
            "categories" : {"$elemMatch":{"$in" : business["categories"]}},
            "is_open" : 1}
    
    close_enough_biznuz = db.businesses.find(pipe)    
    
    print business
    return [close_biz for close_biz in list(close_enough_biznuz) if 
            (gpxpy.geo.haversine_distance(latitude, longitude, close_biz['latitude'], close_biz['longitude'])/1609.34) < r]

In [18]:
competitors = find_competitors_in_region("LTlCaCGZE14GuaUXUGbamg", 5)

36.119922671 36.264645529 -115.248934572 -115.069609028
{u'city': u'Las Vegas', u'neighborhood': u'', u'name': u'Cut and Taste', u'business_id': u'LTlCaCGZE14GuaUXUGbamg', u'longitude': -115.1592718, u'hours': [u'Monday 0:0-0:0', u'Tuesday 0:0-0:0', u'Wednesday 0:0-0:0', u'Thursday 0:0-0:0', u'Friday 0:0-0:0', u'Saturday 0:0-0:0', u'Sunday 0:0-0:0'], u'state': u'NV', u'postal_code': u'89106', u'categories': [u'Caterers', u'Grocery', u'Food', u'Event Planning & Services', u'Party & Event Planning', u'Specialty Food'], u'stars': 5.0, u'address': u'495 S Grand Central Pkwy', u'latitude': 36.1922841, u'review_count': 9, u'_id': ObjectId('58bc80cec541a94ede79e1d5'), u'type': u'business', u'is_open': 1, u'attributes': [u'BusinessAcceptsBitcoin: False', u'BusinessAcceptsCreditCards: True']}


In [20]:
print competitors[10]

{u'city': u'North Las Vegas', u'neighborhood': u'', u'name': u'Dawg Haus', u'business_id': u'FHyLqyxVt-pt6mmOzFbAVg', u'longitude': -115.193509, u'hours': None, u'state': u'NV', u'postal_code': u'89032', u'categories': [u'Street Vendors', u'Food'], u'stars': 4.0, u'address': u'Texas Station', u'latitude': 36.1986381, u'review_count': 4, u'_id': ObjectId('58bc80cec541a94ede79e724'), u'type': u'business', u'is_open': 1, u'attributes': [u'BikeParking: False', u'BusinessAcceptsCreditCards: True', u"BusinessParking: {'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}", u'RestaurantsPriceRange2: 1']}
